the purpose of this notebook is to take the big dataframe created in 07.01-baseline_data_merging, and make an aggregate we can use for our baseline "worst case" scenario model. This will then be fed into R, where we will use the MICE package to impute data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np
import glob

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The 'cachedir' parameter has been deprecated in version 0.12 and will be removed in version 0.14.
You provided "cachedir='/tmp'", use "location='/tmp'" instead.
  # Remove the CWD from sys.path while we load stuff.


In [2]:
#cohort import
from pathlib import Path
os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling') #use to change working directory
wd= os.getcwd() #'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'

date='25012019'
final_pt_df2 = pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/25012019_final_pt_df2_v.csv', index_col=0) #final cohort for PA 

patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 67.5 ms


In [3]:
# #importing in all clinical_variable files
# date= '22112018'
# os.chdir(r'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/processed/')
# big_df= pd.read_csv(Path(wd+'/data/processed/%s_longdf_preImp.csv' %(date),  index_col=0))
# big_df= big_df.reset_index(drop=True).iloc[:,1:]

time: 1.05 ms


In [4]:
#folder to save files to:
save_date= '25012019'
save_path= wd+'/data/processed/PA_project/merged'

def save_df(df, df_name='default',add_prepped=True):
    if df_name == 'default':
        df_name= "%s"%(df)
    
    if add_prepped==False:
        pd.DataFrame(df).to_csv(Path(save_path+'/%s_%s.csv' %(save_date, df_name)))
    else:
        pd.DataFrame(df).to_csv(Path(save_path+'/%s_%s_prepped.csv' %(save_date, df_name)))


time: 6.58 ms


In [5]:
big_df= pd.read_csv(Path(save_path+'/%s_longdf_preImp_prepped.csv' %(date),  index_col=0))
big_df= big_df.reset_index(drop=True).iloc[:,1:]

time: 4.83 s


In [6]:
big_df.head()

,icustay_id,first_pos_else_neg_ssc,label,value,delta,uom,source,subject_id
0,200014.0,2105-02-17 00:00:00,vent_recieved,Mech,-1 days +00:00:00.000000000,mech/O2/none category,ventcategory,9514
1,200014.0,2105-02-17 00:00:00,diasbp,67.0,-1 days +23:16:00.000000000,mmHg,vitals,9514
2,200014.0,2105-02-17 00:00:00,heartrate,71.0,-1 days +23:16:00.000000000,bpm,vitals,9514
3,200014.0,2105-02-17 00:00:00,meanartpress,89.0,-1 days +23:16:00.000000000,mmHg,vitals,9514
4,200014.0,2105-02-17 00:00:00,resprate,10.0,-1 days +23:16:00.000000000,insp/min,vitals,9514


time: 15.4 ms


### 4/1/19 looking at top 5 false pos and false negatives for error analysis to see if the patients have values that would be more predictive within the timewindow (asking if aggregation is the issue).

In [97]:
false_negatives=["222292","207436","290069","257078","279153"]
false_pos=["221378","226860","200065","223926","268739"]
predictors=["lactate","bun","maxSodium","lactate","ph","platelet","temperature","heartrate"]

time: 1.75 ms


In [101]:
def error_investigate(label):
    
    false_negatives=["222292","207436","290069","257078","279153"]
    false_pos=["221378","226860","200065","223926","268739"]

    false_neg_temps=(big_df.loc[
        (big_df.loc[:,"icustay_id"].isin(false_negatives)) &
        (big_df.loc[:,"label"]==label),:]
    ).copy()
    false_pos_temps=(big_df.loc[
        (big_df.loc[:,"icustay_id"].isin(false_pos)) &
        (big_df.loc[:,"label"]==label),:]
    ).copy()
    
    #converting "object" to floats
    #false_neg_temps['value']=false_neg_temps['value'].astype("float")
    false_neg_temps['value']=false_neg_temps['value'].apply(pd.to_numeric)
    #false_pos_temps['value']=false_pos_temps['value'].astype("float")
    false_pos_temps['value']=false_pos_temps['value'].apply(pd.to_numeric)

    #print the descriptive stats of false neg and false pos
    return(print(
        "\n",
        label,
        "\n",
        false_neg_temps.groupby("icustay_id")['value'].describe(percentiles=[0.5]),
        "\n",
        false_pos_temps.groupby("icustay_id")['value'].describe(percentiles=[0.5])
    ))


time: 16.9 ms


In [126]:
#from tabulate import tabulate
#import prettytable
display((false_pos_temps.groupby("icustay_id")['value'].describe(percentiles=[0.5])))#.to_html("table")

,count,mean,std,min,50%,max
icustay_id,,,,,,
200065.0,14.0,36.532540,2.361006,32.222222,36.805556,39.888891
221378.0,44.0,35.814394,0.411538,35.200000,35.711112,36.700000
223926.0,13.0,36.896581,1.120235,35.277778,36.888889,40.000000
226860.0,8.0,38.236111,0.626959,37.166667,38.166667,39.166667
268739.0,9.0,37.233333,0.394053,36.777778,37.111111,37.777778


time: 20.3 ms


In [127]:
predictors=["sodium","lactate","bun","ph","platelet","temperature","heartrate"]
#error_investigate('temperature')
#error_investigate()

for element in predictors:
    error_investigate(element)


 sodium 
             count        mean       std    min    50%    max
icustay_id                                                  
207436.0      2.0  137.500000  0.707107  137.0  137.5  138.0
222292.0      2.0  140.000000  1.414214  139.0  140.0  141.0
257078.0      3.0  138.333333  0.577350  138.0  138.0  139.0
279153.0      3.0  141.000000  2.645751  139.0  140.0  144.0
290069.0      7.0  146.285714  1.889822  144.0  146.0  150.0 
             count        mean       std    min    50%    max
icustay_id                                                  
200065.0     11.0  140.090909  2.119177  138.0  140.0  145.0
221378.0      9.0  141.333333  3.741657  134.0  142.0  148.0
223926.0      3.0  138.000000  1.732051  136.0  139.0  139.0
226860.0      5.0  144.400000  2.607681  142.0  144.0  148.0
268739.0      5.0  140.600000  0.894427  140.0  140.0  142.0

 lactate 
             count      mean       std  min   50%  max
icustay_id                                           
207436.0     

# saving a data dictionary

In [32]:
labeldf=big_df.loc[:,['source','label','uom']]
data_dictionary=labeldf.groupby(['source','label','uom']).size().reset_index()
save_df(data_dictionary, 'pa_data_dictionary', add_prepped=False)
#data_dictionary

time: 562 ms


# qc

In [15]:
big_df.groupby('label')['value'].describe().sort_values('unique')
##categorical variables:
#rrt, phenylephrine, norepinephrine, vasopressin, dobutamine, dopamine, epinephrine, leukocyte, vent_recieved


,count,unique,top,freq
label,,,,
o2_flow,8150,1,0.0,8150
rrt,8629,2,0.0,7820
nitrite,8629,2,Neg/Not_tested,8272
vasopressin,9979,2,0.0,8366
gender,8629,2,M,4937
epinephrine,9277,2,0.0,8507
dopamine,11847,2,0.0,7955
norepinephrine,18757,2,1.0,12115
dobutamine,9172,2,0.0,8464


time: 1.04 s


In [16]:
big_df[big_df['label']=='vent_recieved']['value'].value_counts() #
#df["B"] = df["A"].astype('category')

Mech      4526
Oxygen    3461
None       404
Name: value, dtype: int64

time: 199 ms


In [18]:
big_df.loc[big_df.loc[:,'label']=='vent_recieved','value']= big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].astype('category')

time: 556 ms


In [19]:
#big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].apply(astype('category'))
big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'] = pd.Categorical(big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'])

time: 543 ms


In [20]:
big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].describe()

count     8391
unique       3
top       Mech
freq      4526
Name: value, dtype: object

time: 202 ms


In [21]:
hi_value= ['bands',
'bilirubin',
'bun',
'chloride',
'creatinine',
'daily_sofa',
'glucose',
'heartrate',
'inr',
'lactate',
'pco2',
'potassium',
'ptt',
'resprate',
'temperature',
'weight', 'rrt', 'phenylephrine', 'norepinephrine', 'vasopressin', 'dobutamine', 'dopamine', 'epinephrine',
           'first_admit_age']

low_value=['bicarbonate',
'diasbp',
'hemoglobin',
'meanartpress',
'mingcs',
'ph',
'platelet',
'spo2',
'sysbp',
'pao2/fio2']

both_value=['calcium',
'sodium',
'wbc']

time: 7.3 ms


In [22]:
#lambda x: x if not np.isnan(x) else -1

dict1={str(x):max for x in hi_value}
dict2={str(x):min for x in low_value } #dict2={str(x):min for x in low_value }
dict3={str(x):[max,min] for x in both_value }

time: 2.15 ms


In [23]:
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

dict1 = merge_two_dicts(dict1, dict2)
dict1 = merge_two_dicts(dict1, dict3)

time: 2.82 ms


In [24]:
len(dict1.keys())

37

time: 1.91 ms


### to do: need to convert all dtypes to a numeric type taht pereserves nan. 

In [25]:
categorical=['leukocyte','nitrite','vent_recieved', 'ethnicity','gender']

big_categorical= big_df.loc[big_df.loc[:,'label'].isin(categorical),:]
big_noCat= big_df.loc[big_df.loc[:,'label'].isin(dict1.keys()),:]
big_noCat['value']= big_noCat['value'].apply(pd.to_numeric, args=('coerce',)) #instead of convert to float, may preserve nan's better. 


time: 1min 10s


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [26]:
big_max= big_noCat.loc[big_noCat.loc[:,'label'].isin(hi_value),:]
table = pd.pivot_table(big_max, values='value', columns='label', index=['icustay_id'],aggfunc=max, dropna=False)

time: 655 ms


In [27]:
big_min= big_noCat.loc[big_noCat.loc[:,'label'].isin(low_value),:]
table2 = pd.pivot_table(big_min, values='value', columns='label', index=['icustay_id'],aggfunc=min, dropna=False)

time: 602 ms


In [28]:
big_both= big_noCat.loc[big_noCat.loc[:,'label'].isin(both_value),:]
table3 = pd.pivot_table(big_both,
                        values='value',
                        columns='label',
                        index=['icustay_id'],
                        aggfunc=[max,min],
                        dropna=False)

time: 213 ms


In [29]:
print(len(table),len(table2),len(table3))

8629 8629 8618
time: 2.06 ms


In [30]:
worst_df=pd.merge(table.reset_index(), table2.reset_index(), how='left')
worst_df=pd.merge(worst_df, table3.reset_index(), left_on='icustay_id', right_on='icustay_id',how='left')

time: 78.1 ms


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/generic.py:3111: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [31]:
print(len(list(worst_df)), #40- icustay_id - label = 38. with 

big_df['label'].nunique(),
     
set(big_df['label'].unique())- set(worst_df)   )


41 45 {'sodium', 'leukocyte', 'height', 'wbc', 'o2_flow', 'vent_recieved', 'gender', 'calcium', 'nitrite', 'o2flow', 'ethnicity'}
time: 332 ms


In [32]:
big_max= big_df.loc[big_df.loc[:,'label'].isin(hi_value),:]

time: 384 ms


In [33]:
big_categorical.head()

,icustay_id,first_pos_else_neg_ssc,label,value,delta,uom,source,subject_id
0,200014.0,2105-02-17 00:00:00,vent_recieved,Mech,-1 days +00:00:00.000000000,mech/O2/none category,ventcategory,9514
39,200014.0,2105-02-17 00:00:00,ethnicity,unknown/other,0 days 00:00:00.000000000,NaN,pt_info,9514
41,200014.0,2105-02-17 00:00:00,gender,M,0 days 00:00:00.000000000,NaN,pt_info,9514
302,200014.0,2105-02-17 00:00:00,leukocyte,Neg/Not_tested,1 days 00:00:00.000000000,pos/neg category,leuk,9514
303,200014.0,2105-02-17 00:00:00,nitrite,Neg/Not_tested,1 days 00:00:00.000000000,pos/neg category,nit,9514


time: 13.7 ms


In [34]:
big_categorical= big_categorical.pivot(
    index='icustay_id',
    values='value',
    columns='label') #need to convert to long format. should be one row per patient per time. 

time: 35.7 ms


In [35]:
worst_df=pd.merge(worst_df, big_categorical, left_on='icustay_id', right_on='icustay_id',how='left')

time: 17.2 ms


In [36]:
worst_df['leukocyte']=worst_df['leukocyte'].astype("category")
worst_df['nitrite']=worst_df['nitrite'].astype("category")
worst_df['vent_recieved']=worst_df['vent_recieved'].astype("category")
worst_df['ethnicity']=worst_df['ethnicity'].astype("category")
worst_df['gender']=worst_df['gender'].astype("category")

time: 27.3 ms


In [37]:
list(worst_df)

['icustay_id',
 'bands',
 'bilirubin',
 'bun',
 'chloride',
 'creatinine',
 'daily_sofa',
 'dobutamine',
 'dopamine',
 'epinephrine',
 'first_admit_age',
 'glucose',
 'heartrate',
 'inr',
 'lactate',
 'norepinephrine',
 'pco2',
 'phenylephrine',
 'potassium',
 'ptt',
 'resprate',
 'rrt',
 'temperature',
 'vasopressin',
 'weight',
 'bicarbonate',
 'diasbp',
 'hemoglobin',
 'meanartpress',
 'mingcs',
 'pao2/fio2',
 'ph',
 'platelet',
 'spo2',
 'sysbp',
 ('max', 'calcium'),
 ('max', 'sodium'),
 ('max', 'wbc'),
 ('min', 'calcium'),
 ('min', 'sodium'),
 ('min', 'wbc'),
 'ethnicity',
 'gender',
 'leukocyte',
 'nitrite',
 'vent_recieved']

time: 10.4 ms


In [43]:
worst_df

,icustay_id,bands,bilirubin,bun,chloride,creatinine,daily_sofa,dobutamine,dopamine,epinephrine,...,"(max, sodium)","(max, wbc)","(min, calcium)","(min, sodium)","(min, wbc)",ethnicity,gender,leukocyte,nitrite,vent_recieved
0,200014.0,NaN,0.3,24.0,114.0,0.6,2.0,0.0,0.0,0.0,...,144.0,14.7,1.13,142.0,14.3,unknown/other,M,Neg/Not_tested,Neg/Not_tested,Mech
1,200019.0,NaN,NaN,30.0,109.0,1.1,6.0,0.0,0.0,0.0,...,143.0,12.8,NaN,140.0,11.5,white/nonhispanic,F,Neg/Not_tested,Neg/Not_tested,Mech
2,200030.0,NaN,NaN,28.0,111.0,1.0,4.0,1.0,0.0,0.0,...,141.0,13.9,1.15,140.0,12.9,black,M,Neg/Not_tested,Neg/Not_tested,Oxygen
3,200033.0,NaN,1.4,20.0,105.0,0.9,2.0,0.0,0.0,0.0,...,138.0,6.7,1.03,135.0,4.9,white/nonhispanic,M,Neg/Not_tested,Neg/Not_tested,Mech
4,200055.0,NaN,0.3,40.0,100.0,8.9,8.0,0.0,0.0,0.0,...,140.0,6.3,1.11,137.0,5.7,black,F,Neg/Not_tested,Neg/Not_tested,None
5,200061.0,NaN,5.9,15.0,103.0,0.9,3.0,0.0,0.0,0.0,...,138.0,15.6,NaN,137.0,14.0,unknown/other,M,Neg/Not_tested,Neg/Not_tested,Oxygen
6,200063.0,NaN,NaN,27.0,107.0,2.1,6.0,0.0,0.0,0.0,...,138.0,8.1,1.00,136.0,7.8,unknown/other,M,Neg/Not_tested,Neg/Not_tested,Mech
7,200065.0,40.0,1.3,27.0,114.0,3.1,15.0,0.0,0.0,0.0,...,145.0,56.9,0.54,138.0,15.2,white/nonhispanic,F,Neg/Not_tested,Neg/Not_tested,Mech
8,200075.0,NaN,3.1,12.0,110.0,0.7,8.0,0.0,0.0,0.0,...,139.0,17.3,NaN,135.0,16.4,white/nonhispanic,F,Neg/Not_tested,Neg/Not_tested,Oxygen
9,200078.0,NaN,NaN,15.0,112.0,1.0,3.0,0.0,0.0,0.0,...,136.0,21.9,1.10,132.0,12.3,white/nonhispanic,M,Neg/Not_tested,Neg/Not_tested,Mech


time: 73.5 ms


In [38]:
# baselinePts= final_pt_df2.loc[final_pt_df2.loc[:,'final_bin'].isin(["C_pos/A_full","C_neg/A_partial"]),"icustay_id"]

# worst_df2= worst_df.loc[worst_df.loc[:,'icustay_id'].isin(baselinePts),:] 

time: 25.4 ms


In [40]:
#date= '22112018'

save_df(worst_df, 'worstdf_preImp', add_prepped=False)
#save_df(worst_df2, 'worstdf2_preImp', add_prepped=False)

time: 347 ms


In [42]:
len(worst_df)
worst_df

,icustay_id,bands,bilirubin,bun,chloride,creatinine,daily_sofa,dobutamine,dopamine,epinephrine,...,"(max, sodium)","(max, wbc)","(min, calcium)","(min, sodium)","(min, wbc)",ethnicity,gender,leukocyte,nitrite,vent_recieved
0,200014.0,NaN,0.3,24.0,114.0,0.6,2.0,0.0,0.0,0.0,...,144.0,14.7,1.13,142.0,14.3,unknown/other,M,Neg/Not_tested,Neg/Not_tested,Mech
1,200019.0,NaN,NaN,30.0,109.0,1.1,6.0,0.0,0.0,0.0,...,143.0,12.8,NaN,140.0,11.5,white/nonhispanic,F,Neg/Not_tested,Neg/Not_tested,Mech
2,200030.0,NaN,NaN,28.0,111.0,1.0,4.0,1.0,0.0,0.0,...,141.0,13.9,1.15,140.0,12.9,black,M,Neg/Not_tested,Neg/Not_tested,Oxygen
3,200033.0,NaN,1.4,20.0,105.0,0.9,2.0,0.0,0.0,0.0,...,138.0,6.7,1.03,135.0,4.9,white/nonhispanic,M,Neg/Not_tested,Neg/Not_tested,Mech
4,200055.0,NaN,0.3,40.0,100.0,8.9,8.0,0.0,0.0,0.0,...,140.0,6.3,1.11,137.0,5.7,black,F,Neg/Not_tested,Neg/Not_tested,None
5,200061.0,NaN,5.9,15.0,103.0,0.9,3.0,0.0,0.0,0.0,...,138.0,15.6,NaN,137.0,14.0,unknown/other,M,Neg/Not_tested,Neg/Not_tested,Oxygen
6,200063.0,NaN,NaN,27.0,107.0,2.1,6.0,0.0,0.0,0.0,...,138.0,8.1,1.00,136.0,7.8,unknown/other,M,Neg/Not_tested,Neg/Not_tested,Mech
7,200065.0,40.0,1.3,27.0,114.0,3.1,15.0,0.0,0.0,0.0,...,145.0,56.9,0.54,138.0,15.2,white/nonhispanic,F,Neg/Not_tested,Neg/Not_tested,Mech
8,200075.0,NaN,3.1,12.0,110.0,0.7,8.0,0.0,0.0,0.0,...,139.0,17.3,NaN,135.0,16.4,white/nonhispanic,F,Neg/Not_tested,Neg/Not_tested,Oxygen
9,200078.0,NaN,NaN,15.0,112.0,1.0,3.0,0.0,0.0,0.0,...,136.0,21.9,1.10,132.0,12.3,white/nonhispanic,M,Neg/Not_tested,Neg/Not_tested,Mech


time: 72.6 ms


In [30]:
# big_categorical["icustay_id"].isin(icustay_id)

time: 680 µs


In [31]:
#big_noCat['value']=big_noCat['value'].astype(float, copy=False)
#big_noCat.groupby(['icustay_id','label'], as_index=False)['value'].agg(dict1) #

time: 635 µs


In [32]:
# test= big_noCat.groupby(['icustay_id'], as_index=False)['value'].agg(dict1)
# table = pd.pivot_table(big_noCat, values='value', columns='label', index=['icustay_id'],aggfunc=max, dropna=False)
# table

# pd.pivot_table(df,index=["Manager","Status"],columns=["Product"],values=["Quantity","Price"],
#                aggfunc={"Quantity":len,"Price":np.sum},fill_value=0)

time: 879 µs


In [33]:
# #table = pivot_table(df, values=['D', 'E'], index=['A', 'C'],aggfunc={})
# ##don't know why this doesn't work. 
# table = pd.pivot_table(big_noCat, values='value',columns='label', index='icustay_id',aggfunc=dict1, dropna=False)
# table


time: 1.15 ms


In [34]:
# big_agg= big_df.loc[big_df.loc[:,'label'].isin(non_category),:].groupby(['icustay_id','label'], as_index=False)['value'].apply(lambda x: float(x))

    #.agg({str(x):['max'] for x in hi_value})
#big_agg_count= big_agg.reset_index().pivot(index='icustay_id',columns='label', values=0)#, levels='icustay_id')

time: 713 µs


In [35]:
# hi_df= big_df.loc[big_df.loc[:,'label'].isin(hi_value),:]
# hi_df['value']=hi_df['value'].astype(float, copy=False)
# hi_df.groupby(['icustay_id','label'], as_index=False)['value'].agg('max') #this works

time: 801 µs


In [36]:
# %whos

# del big_agg
# del hi_df
# del final_pt_df2

time: 673 µs
